# Analysis of the Transportation Loads by Balancing Authority

This notebook analyzes the time series of transportation and total loads by Balancing Authority (BA).

In [6]:
# Start by importing the packages we need:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Set the Directory Structure

In [7]:
# Identify the top-level directory and the subdirectory where the data will be stored:
data_input_dir =  '/Users/burl878/Documents/IMMM/GODEEP/Data/Merged_BA_Loads/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/load_analysis/figures/BAs/'


## Make the Plots


In [65]:
# Define a function to plot the comparison between the TELL and transportation loads for a given BA:
def plot_data_comparison(ba_to_plot: str, scenario_to_plot: str, year_to_plot: str, data_input_dir: str, image_output_dir: str, save_images=False):
    
    # Read in the .csv file:
    ba_df = pd.read_csv((data_input_dir + 'Total_Load_Time_Series_' + scenario_to_process + '_' + year_to_process + '.csv'))
    
    # Subset to just the BA you want to plot:
    ba_df = ba_df.loc[ba_df['BA_Code'] == ba_to_plot]
    
    # Convert the timestamp into a datetime variable:
    ba_df['Datetime'] = pd.to_datetime(ba_df['Time_UTC'])
   
    # Use datetime string to get the year, month, day, and hour:
    ba_df['Year'] = ba_df['Datetime'].dt.strftime('%Y').astype(int)
    ba_df['Month'] = ba_df['Datetime'].dt.strftime('%m').astype(int)
    ba_df['Day'] = ba_df['Datetime'].dt.strftime('%d').astype(int)
    ba_df['Hour'] = ba_df['Datetime'].dt.strftime('%H').astype(int)
    
    # Calculate the load ratio (transportation load divided by total load):
    ba_df['Load_Ratio'] = (ba_df['Transportation_Load_MWh']/ba_df['Non-Transportation_Load_MWh']).round(3)
        
    # Compute the maximum load to be used in plotting:
    max_load = ba_df[['Non-Transportation_Load_MWh', 'Transportation_Load_MWh']].max(axis=0).max(axis=0)
    
    # Initiate an empty dataframe to store the mean diurnal cycle results:
    mean_tell_load_df = pd.DataFrame()
    mean_trn_load_df = pd.DataFrame()
    
    # Loop over the seasons and compute the mean diurnal cycle for each season:
    for season in range(4):
        months = pd.DataFrame()
        if season == 0: # Dec-Jan-Feb
           months.loc[1,1] = 12; months.loc[1,2] = 1; months.loc[1,3] = 2;
        if season == 1: # Mar-Apr-May
           months.loc[1,1] = 3; months.loc[1,2] = 4; months.loc[1,3] = 5;
        if season == 2: # Jun-Jul-Aug
           months.loc[1,1] = 6; months.loc[1,2] = 7; months.loc[1,3] = 8;
        if season == 3: # Sep-Oct_Nov
           months.loc[1,1] = 9; months.loc[1,2] = 10; months.loc[1,3] = 11;
            
        # Subset the data to the three months in a given season:
        month_subset_df = ba_df.loc[(ba_df['Month'] == months.loc[1,1]) | (ba_df['Month'] == months.loc[1,2]) | (ba_df['Month'] == months.loc[1,3])]
        
        # Loop over the hours of a day and subset the data to only that hour:
        for hour in range(24):
            # Subset the seasonal data to only that hour:
            hour_subset_df = month_subset_df.loc[(month_subset_df['Hour'] == hour)]
            
            # Compute the mean total and mean transportation load for that season and hour combination:
            mean_tell_load_df.loc[season,hour] = hour_subset_df['Non-Transportation_Load_MWh'].mean()
            mean_trn_load_df.loc[season,hour] = hour_subset_df['Transportation_Load_MWh'].mean()
            
            # Clean up the dataframes we no longer need:
            del hour_subset_df
    
        # Clean up the dataframes we no longer need:
        del months, month_subset_df
    
    # Reset the index:
    ba_df = ba_df.reset_index(drop=True)
    
    # Find the index of the maximum load ratio throughout the year:
    index = ba_df['Load_Ratio'].idxmax(axis=0)
    
    if index > 84:
       start = (index - 84)
    else:
       start = 0

    if index < (len(ba_df)-84):
       end = (index + 84)
    else:
       end = len(ba_df)

    # Create a subset of just the week around the maximum load ratio:
    peak_df = ba_df[start:end]
    
    # Make the plot:
    plt.figure(figsize=(25, 25))
    plt.rcParams['font.size'] = 14
    
    plt.subplot(321)
    plt.scatter(ba_df['Non-Transportation_Load_MWh'], ba_df['Transportation_Load_MWh'], s=15, c='blue')
    plt.plot([0, max_load], [0, max_load], color='k', linestyle='-', linewidth=2)
    plt.plot([0, max_load], [0, (0.9*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.plot([0, max_load], [0, (0.8*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.plot([0, max_load], [0, (0.7*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.plot([0, max_load], [0, (0.6*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.plot([0, max_load], [0, (0.5*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.plot([0, max_load], [0, (0.4*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.plot([0, max_load], [0, (0.3*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.plot([0, max_load], [0, (0.2*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.plot([0, max_load], [0, (0.1*max_load)], color='k', linestyle='-', linewidth=0.5)
    plt.xlim([0, max_load])
    plt.ylim([0, max_load])
    plt.xlabel('Total Non-Transportation Load [MWh]')
    plt.ylabel('Total Transportation Load [MWh]')
    plt.title(year_to_plot + ' ' + scenario_to_plot + ' Transportation vs Non-Transportation Load in ' + ba_to_plot)
    
    plt.subplot(322)
    plt.hist(ba_df['Load_Ratio'], bins = (pd.Series(np.arange(0,2,0.05))))
    plt.xlim([0, 2])
    plt.xlabel('Load Ratio [Transportation/Non-Transportation]')
    plt.ylabel('Hours')
    plt.title(year_to_plot + ' ' + scenario_to_plot + ' Load Ratios in ' + ba_to_plot + ': Min = ' + str(ba_df['Load_Ratio'].min(axis=0).round(3)) + ', Max = ' + str(ba_df['Load_Ratio'].max(axis=0).round(3)))
    
    plt.subplot(312)
    plt.plot(np.arange(0,24,1), mean_tell_load_df.loc[0,:], color = 'b', linewidth = 3, linestyle = '-', label = 'Non-Transportation: DJF')
    plt.plot(np.arange(0,24,1), mean_trn_load_df.loc[0,:],  color = 'b', linewidth = 3, linestyle = ':', label = 'Transportation: DJF')
    plt.plot(np.arange(0,24,1), mean_tell_load_df.loc[1,:], color = 'g', linewidth = 3, linestyle = '-', label = 'Non-Transportation: MAM')
    plt.plot(np.arange(0,24,1), mean_trn_load_df.loc[1,:],  color = 'g', linewidth = 3, linestyle = ':', label = 'Transportation: MAM')
    plt.plot(np.arange(0,24,1), mean_tell_load_df.loc[2,:], color = 'r', linewidth = 3, linestyle = '-', label = 'Non-Transportation: JJA')
    plt.plot(np.arange(0,24,1), mean_trn_load_df.loc[2,:],  color = 'r', linewidth = 3, linestyle = ':', label = 'Transportation: JJA')
    plt.plot(np.arange(0,24,1), mean_tell_load_df.loc[3,:], color = 'orange', linewidth = 3, linestyle = '-', label = 'Non-TransportationL: SON')
    plt.plot(np.arange(0,24,1), mean_trn_load_df.loc[3,:],  color = 'orange', linewidth = 3, linestyle = ':', label = 'Transportation: SON')
    plt.legend()
    plt.xlim([0, 23])
    plt.xlabel('Hour of the Day [UTC]')
    plt.ylabel('Mean Load [MWh]')
    plt.title('Mean Seasonal Diurnal Cycles')
        
    plt.subplot(313)
    plt.plot(peak_df['Time_UTC'], peak_df['Non-Transportation_Load_MWh'], 'k', linewidth=3, label='Total Non-Transportation Load')
    plt.plot(peak_df['Time_UTC'], peak_df['Transportation_Load_MWh'], 'b', linewidth=3, label='Total Transportation Load')
    plt.xlim(peak_df['Time_UTC'].dropna().min(), peak_df['Time_UTC'].dropna().max())
    plt.xticks([])
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Demand [MWh]')
    plt.title('Week of the Highest Load Ratio: ' + peak_df['Time_UTC'].min() + ' to ' + peak_df['Time_UTC'].max())
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       if scenario_to_plot == 'BAU_Climate':
          filename = (ba_to_plot + '_' + 'BAU_Trans_' + year_to_plot + '.png')
       else:
          filename = (ba_to_plot + '_' + 'NetZero_Trans_' + year_to_plot + '.png') 
       plt.savefig(os.path.join(image_output_dir, filename), dpi=75, bbox_inches='tight')
       plt.close()
    

In [68]:
bas = ['AVA', 'AZPS', 'BANC', 'BPAT', 'CHPD', 'CISO', 'DOPD', 'EPE', 'GCPD', 'IID', 'IPCO', 'LDWP', 'NEVP', 'NWMT', 'PACE', 'PACW', 'PGE', 'PNM',
       'PSCO', 'PSEI', 'SCL', 'SRP', 'TEPC', 'TIDC', 'TPWR', 'WACM', 'WALC', 'WAUW']
scenarios = ['BAU_Climate', 'NetZeroNoCCS_Climate']
years = ['2035', '2050']

for i in range(len(bas)):
    for j in range(len(scenarios)):
        for k in range(len(years)):
            plot_data_comparison(ba_to_plot = bas[i],
                                 scenario_to_plot = scenarios[j],
                                 year_to_plot = years[k], 
                                 data_input_dir = data_input_dir, 
                                 image_output_dir = image_output_dir,
                                 save_images = True)
